# K-Nearest Neighbors (K-NN)

### 參考課程實作並在datasets_483_982_spam.csv的資料集中獲得90% 以上的 accuracy (testset)

## Importing the libraries

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
import glob
import codecs
import re

## Importing the dataset

In [7]:
dataset = pd.read_csv(r'datasets_483_982_spam.csv', encoding='latin-1')
dataset['v1'].replace('ham', 0, inplace=True)
dataset['v1'].replace('spam', 1, inplace=True)
print(f"dataset:{dataset}")
all_data = dataset[['v2', 'v1']].values
print(f"all_data:{all_data}")
"讀取資料集"

dataset:      v1                                                 v2 Unnamed: 2  \
0      0  Go until jurong point, crazy.. Available only ...        NaN   
1      0                      Ok lar... Joking wif u oni...        NaN   
2      1  Free entry in 2 a wkly comp to win FA Cup fina...        NaN   
3      0  U dun say so early hor... U c already then say...        NaN   
4      0  Nah I don't think he goes to usf, he lives aro...        NaN   
...   ..                                                ...        ...   
5567   1  This is the 2nd time we have tried 2 contact u...        NaN   
5568   0              Will Ì_ b going to esplanade fr home?        NaN   
5569   0  Pity, * was in mood for that. So...any other s...        NaN   
5570   0  The guy did some bitching but I acted like i'd...        NaN   
5571   0                         Rofl. Its true to its name        NaN   

     Unnamed: 3 Unnamed: 4  
0           NaN        NaN  
1           NaN        NaN  
2           NaN 

'讀取資料集'

### 取出訓練內文與標註

In [16]:
X = all_data[:,0]
Y = all_data[:,1].astype(np.uint8)

In [10]:
print('Training Data Examples : \n{}'.format(X[:5]))

Training Data Examples : 
['Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...'
 'Ok lar... Joking wif u oni...'
 "Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's"
 'U dun say so early hor... U c already then say...'
 "Nah I don't think he goes to usf, he lives around here though"]


In [11]:
print('Labeling Data Examples : \n{}'.format(Y[:5]))

Labeling Data Examples : 
[0 0 1 0 0]


### 文字預處理

In [17]:
from sklearn.metrics import confusion_matrix
from nltk.corpus import stopwords

import nltk

#nltk.download('stopwords')

# Lemmatize with POS Tag
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer 

# Create a Lemmatizer
lemmatizer = WordNetLemmatizer() 
def get_wordnet_pos(word):  # From Day 13: Stemming and Lemmatization
    """將pos_tag結果mapping到lemmatizer中pos的格式"""
    print(nltk.pos_tag([word]))
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}

    return tag_dict.get(tag, wordnet.NOUN)  # Return NOUN if no matches

def clean_content(X):  # From Day 14: Word Preprocessing
    # Remove non-alphabet characters
    X_clean = [re.sub('[^a-zA-Z]',' ', x).lower() for x in X]
    # Tokenize
    X_word_tokenize = [nltk.word_tokenize(x) for x in X_clean]
    # Stopwords_lemmatizer
    X_stopwords_lemmatizer = []
    stop_words = set(stopwords.words('english'))
    for content in X_word_tokenize:
        content_clean = []
        for word in content:
            if word not in stop_words:  # Remove redundant words in a content list
                word = lemmatizer.lemmatize(word, get_wordnet_pos(word))
                content_clean.append(word)
        X_stopwords_lemmatizer.append(content_clean)
    
    X_output = [' '.join(x) for x in X_stopwords_lemmatizer]
    
    return X_output

In [18]:
X = clean_content(X)

[('go', 'VB')]
[('jurong', 'NN')]
[('point', 'NN')]
[('crazy', 'NN')]
[('available', 'JJ')]
[('bugis', 'NN')]
[('n', 'NN')]
[('great', 'JJ')]
[('world', 'NN')]
[('la', 'NN')]
[('e', 'NN')]
[('buffet', 'NN')]
[('cine', 'NN')]
[('got', 'VBD')]
[('amore', 'NN')]
[('wat', 'NN')]
[('ok', 'NN')]
[('lar', 'NN')]
[('joking', 'NN')]
[('wif', 'NN')]
[('u', 'NN')]
[('oni', 'NN')]
[('free', 'JJ')]
[('entry', 'NN')]
[('wkly', 'NN')]
[('comp', 'NN')]
[('win', 'NN')]
[('fa', 'NN')]
[('cup', 'NN')]
[('final', 'JJ')]
[('tkts', 'NN')]
[('st', 'NN')]
[('may', 'MD')]
[('text', 'NN')]
[('fa', 'NN')]
[('receive', 'NN')]
[('entry', 'NN')]
[('question', 'NN')]
[('std', 'NN')]
[('txt', 'NN')]
[('rate', 'NN')]
[('c', 'NNS')]
[('apply', 'VB')]
[('u', 'NN')]
[('dun', 'NN')]
[('say', 'VB')]
[('early', 'RB')]
[('hor', 'NN')]
[('u', 'NN')]
[('c', 'NNS')]
[('already', 'RB')]
[('say', 'VB')]
[('nah', 'NN')]
[('think', 'NN')]
[('goes', 'VBZ')]
[('usf', 'NN')]
[('lives', 'NNS')]
[('around', 'IN')]
[('though', 'IN')]
[('

[('crashing', 'VBG')]
[('place', 'NN')]
[('tonight', 'NN')]
[('wow', 'NN')]
[('never', 'RB')]
[('realized', 'VBN')]
[('embarassed', 'VBN')]
[('accomodations', 'NNS')]
[('thought', 'NN')]
[('liked', 'VBN')]
[('since', 'IN')]
[('best', 'JJS')]
[('could', 'MD')]
[('always', 'RB')]
[('seemed', 'VBD')]
[('happy', 'JJ')]
[('cave', 'NN')]
[('sorry', 'NN')]
[('give', 'VB')]
[('sorry', 'NN')]
[('offered', 'VBN')]
[('sorry', 'NN')]
[('room', 'NN')]
[('embarassing', 'VBG')]
[('sms', 'NNS')]
[('ac', 'NN')]
[('sptv', 'NN')]
[('new', 'JJ')]
[('jersey', 'NN')]
[('devils', 'NNS')]
[('detroit', 'NN')]
[('red', 'JJ')]
[('wings', 'NNS')]
[('play', 'NN')]
[('ice', 'NN')]
[('hockey', 'NN')]
[('correct', 'NN')]
[('incorrect', 'NN')]
[('end', 'NN')]
[('reply', 'NN')]
[('end', 'NN')]
[('sptv', 'NN')]
[('know', 'VB')]
[('mallika', 'NN')]
[('sherawat', 'NN')]
[('yesterday', 'NN')]
[('find', 'VB')]
[('lt', 'NN')]
[('url', 'NN')]
[('gt', 'NN')]
[('congrats', 'NNS')]
[('year', 'NN')]
[('special', 'JJ')]
[('cinema'

[('much', 'JJ')]
[('n', 'NN')]
[('puttin', 'NN')]
[('weight', 'NN')]
[('haha', 'NN')]
[('anythin', 'NN')]
[('special', 'JJ')]
[('happened', 'VBD')]
[('entered', 'VBN')]
[('cabin', 'NN')]
[('pa', 'NN')]
[('said', 'VBD')]
[('happy', 'JJ')]
[('b', 'NN')]
[('day', 'NN')]
[('boss', 'NN')]
[('felt', 'NN')]
[('special', 'JJ')]
[('askd', 'NN')]
[('lunch', 'NN')]
[('lunch', 'NN')]
[('invited', 'VBN')]
[('apartment', 'NN')]
[('went', 'VBD')]
[('winner', 'NN')]
[('u', 'NN')]
[('specially', 'RB')]
[('selected', 'VBN')]
[('receive', 'NN')]
[('holiday', 'NN')]
[('flights', 'NNS')]
[('inc', 'NN')]
[('speak', 'NN')]
[('live', 'JJ')]
[('operator', 'NN')]
[('claim', 'NN')]
[('p', 'NN')]
[('min', 'NN')]
[('goodo', 'NN')]
[('yes', 'NNS')]
[('must', 'MD')]
[('speak', 'NN')]
[('friday', 'NN')]
[('egg', 'NN')]
[('potato', 'NN')]
[('ratio', 'NN')]
[('tortilla', 'NN')]
[('needed', 'VBN')]
[('hmm', 'NN')]
[('uncle', 'NN')]
[('informed', 'VBN')]
[('paying', 'VBG')]
[('school', 'NN')]
[('directly', 'RB')]
[('pls'

[('one', 'CD')]
[('great', 'JJ')]
[('escape', 'NN')]
[('fancy', 'NN')]
[('bridge', 'NN')]
[('needs', 'NNS')]
[('lager', 'NN')]
[('see', 'VB')]
[('tomo', 'NN')]
[('yes', 'NNS')]
[('completely', 'RB')]
[('form', 'NN')]
[('clark', 'NN')]
[('also', 'RB')]
[('utter', 'NN')]
[('waste', 'NN')]
[('sir', 'NN')]
[('need', 'NN')]
[('axis', 'NN')]
[('bank', 'NN')]
[('account', 'NN')]
[('bank', 'NN')]
[('address', 'NN')]
[('hmmm', 'NN')]
[('thk', 'NN')]
[('sure', 'NN')]
[('got', 'VBD')]
[('time', 'NN')]
[('hop', 'NN')]
[('ard', 'NN')]
[('ya', 'NN')]
[('go', 'VB')]
[('free', 'JJ')]
[('abt', 'NN')]
[('muz', 'NN')]
[('call', 'NN')]
[('u', 'NN')]
[('discuss', 'NN')]
[('liao', 'NN')]
[('time', 'NN')]
[('coming', 'VBG')]
[('later', 'RB')]
[('bloody', 'NN')]
[('hell', 'NN')]
[('cant', 'NN')]
[('believe', 'VB')]
[('forgot', 'NN')]
[('surname', 'NN')]
[('mr', 'NN')]
[('ill', 'NN')]
[('give', 'VB')]
[('u', 'NN')]
[('clue', 'NN')]
[('spanish', 'JJ')]
[('begins', 'NNS')]
[('well', 'RB')]
[('gon', 'NN')]
[('na'

[('road', 'NN')]
[('turn', 'NN')]
[('first', 'RB')]
[('right', 'NN')]
[('dear', 'NN')]
[('going', 'VBG')]
[('rubber', 'NN')]
[('place', 'NN')]
[('sorry', 'NN')]
[('battery', 'NN')]
[('died', 'VBD')]
[('yeah', 'NN')]
[('yes', 'NNS')]
[('tv', 'NN')]
[('always', 'RB')]
[('available', 'JJ')]
[('work', 'NN')]
[('place', 'NN')]
[('text', 'NN')]
[('meet', 'NN')]
[('someone', 'NN')]
[('sexy', 'NN')]
[('today', 'NN')]
[('u', 'NN')]
[('find', 'VB')]
[('date', 'NN')]
[('even', 'RB')]
[('flirt', 'NN')]
[('u', 'NN')]
[('join', 'NN')]
[('p', 'NN')]
[('reply', 'NN')]
[('name', 'NN')]
[('age', 'NN')]
[('eg', 'NN')]
[('sam', 'NN')]
[('msg', 'NN')]
[('recd', 'NN')]
[('thirtyeight', 'NN')]
[('pence', 'NN')]
[('printed', 'VBN')]
[('oh', 'NN')]
[('lt', 'NN')]
[('gt', 'NN')]
[('come', 'VB')]
[('upstairs', 'NNS')]
[('ill', 'NN')]
[('little', 'JJ')]
[('closer', 'NN')]
[('like', 'IN')]
[('bus', 'NN')]
[('stop', 'NN')]
[('street', 'NN')]
[('wil', 'NN')]
[('reach', 'NN')]
[('new', 'JJ')]
[('theory', 'NN')]
[('ar

...

[('give', 'VB')]
[('feb', 'NN')]
[('got', 'VBD')]
[('ur', 'NN')]
[('favorite', 'NN')]
[('oyster', 'NN')]
[('n', 'NN')]
[('got', 'VBD')]
[('favorite', 'NN')]
[('sashimi', 'NN')]
[('ok', 'NN')]
[('lar', 'NN')]
[('dun', 'NN')]
[('say', 'VB')]
[('already', 'RB')]
[('wait', 'NN')]
[('ur', 'NN')]
[('stomach', 'NN')]
[('start', 'NN')]
[('rumbling', 'VBG')]
[('sister', 'NN')]
[('going', 'VBG')]
[('earn', 'NN')]
[('da', 'NN')]
[('get', 'VB')]
[('official', 'NN')]
[('england', 'NN')]
[('poly', 'NN')]
[('ringtone', 'NN')]
[('colour', 'NN')]
[('flag', 'NN')]
[('yer', 'NN')]
[('mobile', 'NN')]
[('tonights', 'NNS')]
[('game', 'NN')]
[('text', 'NN')]
[('tone', 'NN')]
[('flag', 'NN')]
[('optout', 'NN')]
[('txt', 'NN')]
[('eng', 'NN')]
[('stop', 'NN')]
[('box', 'NN')]
[('w', 'NN')]
[('wx', 'NN')]
[('hahaha', 'NN')]
[('use', 'NN')]
[('brain', 'NN')]
[('dear', 'NN')]
[('jus', 'NN')]
[('finish', 'NN')]
[('watching', 'VBG')]
[('tv', 'NN')]
[('u', 'NN')]
[('k', 'NN')]
[('fyi', 'NN')]
[('back', 'RB')]
[('par

[('ese', 'JJ')]
[('tihs', 'NN')]
[('msg', 'NN')]
[('evrey', 'NN')]
[('splleing', 'VBG')]
[('tihs', 'NN')]
[('msg', 'NN')]
[('wrnog', 'NN')]
[('bt', 'NN')]
[('sitll', 'NN')]
[('yuo', 'NN')]
[('raed', 'NN')]
[('wihtuot', 'NN')]
[('ayn', 'NN')]
[('mitsake', 'NN')]
[('goodnight', 'NN')]
[('amp', 'NN')]
[('nice', 'JJ')]
[('sleep', 'NN')]
[('sweet', 'NN')]
[('dreams', 'NNS')]
[('cant', 'NN')]
[('pick', 'NN')]
[('phone', 'NN')]
[('right', 'NN')]
[('pls', 'NN')]
[('send', 'NN')]
[('message', 'NN')]
[('want', 'NN')]
[('leave', 'VB')]
[('barely', 'RB')]
[('stay', 'NN')]
[('sane', 'NN')]
[('fighting', 'VBG')]
[('constantly', 'RB')]
[('helping', 'VBG')]
[('current', 'JJ')]
[('leading', 'VBG')]
[('bid', 'NN')]
[('pause', 'NN')]
[('auction', 'NN')]
[('send', 'NN')]
[('customer', 'NN')]
[('care', 'NN')]
[('free', 'JJ')]
[('entry', 'NN')]
[('gr', 'NN')]
[('prizes', 'NNS')]
[('wkly', 'NN')]
[('comp', 'NN')]
[('chance', 'NN')]
[('win', 'NN')]
[('latest', 'JJS')]
[('nokia', 'NNS')]
[('psp', 'NN')]
[('cas

[('hill', 'NN')]
[('fun', 'NN')]
[('night', 'NN')]
[('x', 'NN')]
[('thought', 'NN')]
[('get', 'VB')]
[('watch', 'NN')]
[('cos', 'NN')]
[('thats', 'NNS')]
[('kind', 'NN')]
[('thing', 'NN')]
[('u', 'NN')]
[('get', 'VB')]
[('th', 'NN')]
[('loves', 'NNS')]
[('much', 'JJ')]
[('guaranteed', 'NN')]
[('award', 'NN')]
[('maybe', 'RB')]
[('even', 'RB')]
[('cash', 'NN')]
[('claim', 'NN')]
[('ur', 'NN')]
[('award', 'NN')]
[('call', 'NN')]
[('free', 'JJ')]
[('legitimat', 'NN')]
[('efreefone', 'NN')]
[('number', 'NN')]
[('wat', 'NN')]
[('u', 'NN')]
[('think', 'NN')]
[('good', 'JJ')]
[('morning', 'NN')]
[('repair', 'NN')]
[('shop', 'NN')]
[('reason', 'NN')]
[('hour', 'NN')]
[('fine', 'NN')]
[('got', 'VBD')]
[('enough', 'RB')]
[('bud', 'NN')]
[('last', 'JJ')]
[('night', 'NN')]
[('least', 'JJS')]
[('back', 'RB')]
[('good', 'JJ')]
[('journey', 'NN')]
[('let', 'VB')]
[('know', 'VB')]
[('need', 'NN')]
[('receipts', 'NNS')]
[('shall', 'MD')]
[('tell', 'NN')]
[('like', 'IN')]
[('pendent', 'NN')]
[('takes', 

[('one', 'CD')]
[('law', 'NN')]
[('never', 'RB')]
[('make', 'VB')]
[('ur', 'NN')]
[('friend', 'NN')]
[('feel', 'NN')]
[('alone', 'RB')]
[('alive', 'JJ')]
[('gud', 'NN')]
[('night', 'NN')]
[('private', 'JJ')]
[('account', 'NN')]
[('statement', 'NN')]
[('shows', 'NNS')]
[('un', 'NN')]
[('redeemed', 'VBN')]
[('points', 'NNS')]
[('call', 'NN')]
[('identifier', 'NN')]
[('code', 'NN')]
[('expires', 'NNS')]
[('apo', 'NN')]
[('mokka', 'NN')]
[('players', 'NNS')]
[('perhaps', 'RB')]
[('much', 'JJ')]
[('easy', 'JJ')]
[('give', 'VB')]
[('account', 'NN')]
[('identification', 'NN')]
[('tomorrow', 'NN')]
[('uni', 'JJ')]
[('wait', 'NN')]
[('msg', 'NN')]
[('lt', 'NN')]
[('gt', 'NN')]
[('min', 'NN')]
[('told', 'NN')]
[('tell', 'NN')]
[('stupid', 'JJ')]
[('hear', 'NN')]
[('wont', 'NN')]
[('tell', 'NN')]
[('anything', 'NN')]
[('dad', 'NN')]
[('called', 'VBN')]
[('brother', 'NN')]
[('spoken', 'NN')]
[('god', 'NN')]
[('love', 'NN')]
[('limit', 'NN')]
[('god', 'NN')]
[('grace', 'NN')]
[('measure', 'NN')]
[(

[('next', 'JJ')]
[('lt', 'NN')]
[('gt', 'NN')]
[('hours', 'NNS')]
[('imma', 'NN')]
[('flip', 'NN')]
[('shit', 'NN')]
[('anything', 'NN')]
[('lor', 'NN')]
[('juz', 'NN')]
[('us', 'PRP')]
[('lor', 'NN')]
[('get', 'VB')]
[('dump', 'NN')]
[('heap', 'NN')]
[('mom', 'NN')]
[('decided', 'VBD')]
[('come', 'VB')]
[('lowes', 'NN')]
[('boring', 'NN')]
[('ok', 'NN')]
[('lor', 'NN')]
[('sony', 'NN')]
[('ericsson', 'NN')]
[('salesman', 'NN')]
[('ask', 'NN')]
[('shuhui', 'NN')]
[('say', 'VB')]
[('quite', 'RB')]
[('gd', 'NN')]
[('use', 'NN')]
[('considering', 'VBG')]
[('ard', 'NN')]
[('like', 'IN')]
[('dat', 'NN')]
[('lor', 'NN')]
[('wait', 'NN')]
[('til', 'NN')]
[('least', 'JJS')]
[('wednesday', 'NN')]
[('see', 'VB')]
[('get', 'VB')]
[('huh', 'NN')]
[('lei', 'NN')]
[('reminder', 'NN')]
[('get', 'VB')]
[('pounds', 'NNS')]
[('free', 'JJ')]
[('call', 'NN')]
[('credit', 'NN')]
[('details', 'NNS')]
[('great', 'JJ')]
[('offers', 'NNS')]
[('pls', 'NN')]
[('reply', 'NN')]
[('text', 'NN')]
[('valid', 'JJ')]
[

### Bag of words

In [19]:
from sklearn.feature_extraction.text import CountVectorizer
# max_features是要建造幾個column，會按造字出現的高低去篩選 
cv = CountVectorizer(max_features = 10000)
X = cv.fit_transform(X).toarray()
print(f"After CountVectorizer transformation, X:{X}")

After CountVectorizer transformation, X:[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


In [20]:
X.shape

(5572, 6517)

## Splitting the dataset into the Training set and Test set

In [21]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=0)

## Training the K-NN model on the Training set

In [31]:
from sklearn.neighbors import KNeighborsClassifier
classifier = KNeighborsClassifier(n_neighbors=3, metric='minkowski', p=2)  # 'minkowski' is equivalent to Euclidean Distance when p is 2
classifier.fit(X_train, y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=3, p=2,
                     weights='uniform')

## Predicting a new result

In [23]:
print('Trainset Accuracy: {}'.format(classifier.score(X_train, y_train)))

Trainset Accuracy: 0.9591653578640341


In [32]:
#print('Trainset Accuracy: {}'.format(classifier.score(X_train, y_train)))

Trainset Accuracy: 0.9421135292797846


In [24]:
print('Testset Accuracy: {}'.format(classifier.score(X_test, y_test)))

Testset Accuracy: 0.9282511210762332


In [33]:
#print('Testset Accuracy: {}'.format(classifier.score(X_test, y_test)))

Testset Accuracy: 0.9121076233183857


## Predicting the Test set results

In [25]:
y_pred = classifier.predict(X_test)
print(f"y_pred:{y_pred}")

y_pred:[0 0 0 ... 0 0 0]


## Making the Confusion Matrix

In [26]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)

[[949   0]
 [ 80  86]]


0.9282511210762332